In [ ]:
!pip install plotly -qqq
!pip install scikit-learn -qqq

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

features = pd.read_csv('/kaggle/input/retaildataset/Features data set.csv')
sales = pd.read_csv('/kaggle/input/retaildataset/sales data-set.csv')
stores = pd.read_csv('/kaggle/input/retaildataset/stores data-set.csv')

In [ ]:
data = pd.merge(sales,features, on = ['Store','Date',"IsHoliday"], how = 'left')
data = pd.merge(data, stores, on = ['Store'], how = 'left')
data = data.fillna(0)

In [ ]:
data['Date'] = pd.to_datetime(data['Date'], format = '%d/%m/%Y')
min(data['Date']), max(data['Date'])

Veri Şubat 2010 ile Ekim 2012 Ayları arasında Mağazaların her hafta yaptığı satışları ve bunun yanı sıra mağazaların bulunduğu zaman ve konumda Benzin fiyatı, İşsizlik oranı, Mevcut Haftanın bayram olup olmaması ve de o dönemin TÜFE'sini yani Tüketicilerin belirli bir dönemde olan satın alma gücünü temsil eden bir istatistiği barındırmakta. Bu verileri inceleyeceğiz ve daha sonrasında ise İstatistksel tahmin modelleri ile Mevsimselliği, Trendleri ve Sonraki zamanlardaki Satış tahminleri ortaya çıkarmaya çalışacağım.

In [ ]:
num_columns = [column for column in data.columns if data[column].dtype != 'object']
corr_df = data[num_columns]

corrs = corr_df.corr(method = 'pearson')
fig = px.imshow(corrs, text_auto = True, aspect = 'auto')
fig.show()

In [ ]:
sorted_corr = corrs.unstack().sort_values(ascending = False)
sorted_corr = sorted_corr[(sorted_corr < 1) & (sorted_corr >= 0.1)]
sorted_corr.drop_duplicates(inplace = True)
sorted_corr

Markdown 1-5 Arası Anonim olarak yapılan promosyonları belirtir yani belirli bir üründe veya mağazada yapılan bir promosyonu. Markdown 1 ve 4 arasındak çok yüksek korelasyon mevcut, bunlar birbiri ile bağlantılı promosyonlar olmalı.
Benzin fiyatı ve Tarih parametreleri arasındaki korelasyon da çok yüksek benzin fiyatları oynak olsa gerek. isHoliday ve Markdown-3 parametreleri arasındaki korelasyon sadece özel gün ve bayramlarda yapılan bir promosyona ve indirime işaret ediyor olabilir. Haftalık Satış ve Mağaza büyüklüğü arasında da diğerlerine nazaran daha az da olsa bir korelasyon mevcut yani mağazanın büyüklüğü haftalık satışlara çok da olmasa etki ediyor olabilir. TÜFE ile Sıcaklık arasındaki korelasyon çok garip geldi, bir şekilde az da olsa birbirleri ile bağlantıya sahipler. Yani işsizliğin artış gösteriği yerde sıcaklık da artış gösteriyor.
Sıcaklık ve Tarih arasındaki ilişki zaten çok kaçınılmaz, Tarihin benzin fiyatı ile de ciddi bir korelasyonu bulunduğu için bu onun Sıcaklık ile de en az Tarih kadar korelasyona sahip kılıyor.
Mağaza büyüklüğünün belirli Markdownlarla olan korelasyonu bu Markdownların stok bitirme veya daha büyük bir mağazada uygulanabilen promosyonlar olabileceğini dair düşündürüyor.

In [ ]:
monthly_sales = data.groupby(pd.Grouper(key = 'Date', freq = 'M'))['Weekly_Sales'].sum().round(2)
monthly_sales = monthly_sales.astype('int64')
fig = px.line(x = list(monthly_sales.index.to_list()), y = list(monthly_sales.values), title = 'Total Sales by Monthly')
fig.update_xaxes(title_text = 'Date')
fig.update_yaxes(title_text = 'Total Sales in $')
fig.update_traces(mode = 'markers+lines')
fig.show()

Mağazaların ve Departmanların aylık toplam satış rakamların incelediğimizde aslında bir mevsimellik gözümüze çarpıyor. Yılın ilk ayına girerken ve yılın ilk ayında çok yüksek satış rakamları elde ederken yılın geri kalan diğer aylarında dalgalı ama belirli bir deseni barındıran satış rakamları görüyoruz.

In [ ]:
monthly_fuel = data.groupby(pd.Grouper(key = 'Date', freq = 'M'))['Fuel_Price'].mean().round(2)
monthly_fuel = monthly_fuel.astype('float')
fig = px.line(x = list(monthly_fuel.index.to_list()), y = list(monthly_fuel.values), title = 'Average Fuel Price by Monthly')
fig.update_xaxes(title_text = 'Date')
fig.update_yaxes(title_text = 'Avg. Fuel Price in $')
fig.update_traces(mode = 'markers+lines')
fig.show()

Benzin fiyatları da satışların yüksek olduğu dönemlerde yüksek olduğu dönemlerde az olurken satışların az ve dalgalı olduğu dönemlerde yükselme göstermiş bu da iki parametre arasındaki negatif korelasyonu açıklıyor ama bence satışların çok olduğu dönemlerde benzinin tüketiciler için ucuz olması pozitif bir korelasyon. Böyle bir parametrenin, benzinin ucuz olması gibi bir düşüncenin bile bir tüketiciyi daha sık alışveriş yapma konusunda ikna edebileceğine inanıyorum. Özellikle ABD gibi her yere araba ile gidilen ülkelerde.

In [ ]:
temp_df = data[(data['Store'] == 1) & (data['Dept'] == 1)]
monthly_holiday = temp_df.groupby(pd.Grouper(key = 'Date', freq = 'M'))['IsHoliday'].sum()
monthly_holiday

Bir yıl içerisinde 10 hafta özel ve bayram haftası ilan edilmiş. Önceki satış grafiğine ufak bir göz attığımızda ise özel haftaların satışlara etki ettiği görülebilmekte. Özellike Aralık ve Yılbaşı kutlamaları ve Christmas Yılın başındaki satışlara fazlasıyla etki etmiş.

In [ ]:
data[['Weekly_Sales', 'IsHoliday']].corr(method = 'pearson')

In [ ]:
corr_month_sales_holiday = pd.merge(monthly_sales, monthly_holiday, how = 'left', on = 'Date')
corr_month_sales_holiday.corr(method = 'pearson')

Haftalık satışların bayram parametresi ile korelasyonuna baktığımızda çok düşük bir değer görüyoruz ama aylık olarak bütün mağazaların ve departmanlarının total satışları bazında bayram paratmetresi ile korelasyonunu incelediğimizde %21'lik iyi bir korelasyonu görüyoruz. Korelasyonda perspektifte önemli galiba.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(monthly_sales.index.to_list()),
    y=list(monthly_sales.values),
    mode='markers+lines',
    marker=dict(color=list(monthly_holiday.values))
))

fig.update_layout(height=450)
fig.show()

Kolerasyonun neden daha fazla değil de %21 olduğunu yukarıdaki grafikte daha rahat anlayabiliriz. Özel haftaların bulunduğu aylar sarı nokta ile işaretli. Baktığımızda yükselen aylarda sarı noktaları farkettiğimiz gibi düşen ve düşük olan aylar da sarı noktalar farkediyor veya yükselen aylarda herhangi bir özel hafta belirtmeyen mavi noktayı görebiliyoruz. Bu da korelasyonun %21 olmasının daha rahat anlaşılabilir bir gösterimi gibi.

In [ ]:
graph_columns = ['Temperature','Fuel_Price','Weekly_Sales', 'CPI', 'Unemployment', 'Size']


for col in graph_columns:
    temp_data = data.groupby(pd.Grouper(key = 'Date', freq = 'M'))[col].mean().round(0)
    fig = px.bar(x = temp_data.index.to_list(), y = list(temp_data.values))
    fig.update_xaxes(title_text = 'Dates')
    fig.update_yaxes(title_text = str(col))
    fig.show()

Yukarıda birçok nümerik parametrelerin aylık zaman içerisindeki ortalama değerleri grafiğe dökülmüştür. Grafiklerin birkaçı hariç diğerlerine baktığımızda sahip oldukları desen göze çarpıyor, buna mevsimsellik deniyor. Parametreler belirli dönemler düşme eğilimi gösterirken, belirli dönemler yükselme eğilimi gösteriyor. 

In [ ]:
temp_data = data.groupby(pd.Grouper(key = 'Date', freq = 'M'))['Weekly_Sales'].sum().astype('int').reset_index()
temp_data.set_index('Date')

train = temp_data.loc[:23, :]
test = temp_data.loc[24:, :]

test = test.set_index('Date')

In [ ]:
!pip install statsmodels -qqq

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

y = train['Weekly_Sales']

model = SARIMAX(y, order = (5,0,5))
model = model.fit()

In [ ]:
y_pred = model.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05) 
y_pred_df["Predictions"] = model.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_out = y_pred_df["Predictions"]

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

y_pred_out = y_pred_out.astype('int')
arma_rmse = np.sqrt(mean_squared_error(test["Weekly_Sales"].values, y_pred_df["Predictions"]))
print("ARMA RMSE: ",arma_rmse)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(y_pred_out, color='Blue', label = 'SARIMA Predictions')
plt.plot(test, color = 'Red', label = 'Test Values')
plt.rcParams['figure.figsize'] = [9, 5]
plt.legend()

MSE Metriği çok yüksek yani model sonuçları gerçek test verileri çok uyumsuz. Bunu zaten gerçek test değerlerinin dalgalı şekilde giderken model tahminlerinin linear bir çizgi halinde görünmesinden anlayabilirsiniz. Modelin parametreleri ile biraz oynayalım.

In [ ]:
time_range = pd.date_range(start = '2012-02-25', end = '2013-02-25', freq = 'M')
year_pred = model.get_forecast(len(time_range.to_list()))
year_pred = year_pred.conf_int(alpha = 0.05) 
year_pred["Predictions"] = model.predict(start = year_pred.index[0], end = year_pred.index[-1])
year_pred.index = time_range.to_list()
year_pred_out = year_pred['Predictions']
year_pred_out = year_pred_out.astype('int')

In [ ]:
!pip install seaborn -qqq

In [ ]:
import seaborn as sns
sns.set_theme()

sns.lineplot(test)
sns.lineplot(year_pred_out, label = 'Predictions')
plt.xlabel('Date')
plt.ylabel('Total Sales in Million $')
plt.show()

Yukarıdaki grafikte ise veri setindeki son ay olan 2012 Ekimden 2013 Şubatına kadar aylık satış hacmini tahmin etmeye çalıştık. SARIMA modelimiz Ekimden Kasıma kadar satış hacminde yükselişe giderken Aralık ve Ocak aylarında satışların düşeceğini tahmin etmiş. Modelin tahmini verisetindeki mevsimsellik ile uyumlu 2011 ve 2010 yıllarında da Ekimden Kasıma kadar yükseliş ve Aralıktan Ocağa kadar düşüş gözüküyor.

In [ ]:
!pip install prophet -qqq

In [ ]:
from prophet import Prophet
from prophet.plot import plot_plotly
import plotly.offline as py
py.init_notebook_mode()

In [ ]:
temp_data = temp_data.rename(columns = {
    'Date':'ds',
    'Weekly_Sales':'y'
})

In [ ]:
%matplotlib inline
plt.style.use('fivethirtyeight')

In [ ]:
ax = temp_data.set_index('ds').plot(figsize = (10,4))
ax.set_xlabel('Date')
ax.set_ylabel('Total Sales in Month')

plt.show()

In [ ]:
prop_model = Prophet(interval_width = 0.95)
prop_model.fit(temp_data)

In [ ]:
future_dates = prop_model.make_future_dataframe(periods = 36, freq = 'MS')
forecast = prop_model.predict(future_dates)
forecast_lookup = forecast[['ds','yhat','yhat_lower','yhat_upper']].head(3)
forecast_lookup['yhat'] = forecast['yhat'].astype('int')
forecast_lookup.head(3)

In [ ]:
fig = prop_model.plot(forecast, uncertainty = True, figsize = (10,5))

In [ ]:
fig1 = prop_model.plot_components(forecast, figsize = (8,5))

Aylık Toplam Gelir her sene yükselme trendi gösteriyor. İkinci grafikteki ilk grafikte Trend çizgisinin doğrusal olarak arttığını görebiliyoruz. İkinci grafiğin ikinci grafiğinde ise satışların Nisan, Ağustos ve Ekim aylarında yüksek derecede arttığını görüyoruz diğer aylarda ise dalgalı bir gidişata sahip. Yılın ilk ayları düşüşlerle geçiyor gibi ve tabi Mayıs ve Temmuz aylarında da çok keskin ve sert düşüşler mevcut. Genel olarak en verimli aylar Nisan, Ağustos ve Ekim gibi.

In [ ]:
from prophet.plot import add_changepoints_to_plot

fig = prop_model.plot(forecast, figsize = (11, 4))
a = add_changepoints_to_plot(fig.gca(), prop_model, forecast)

In [ ]:
prop_model.changepoints

Changepoints bir zaman serisindeki kırılma noktalarıdır. Elimizdeki aylık toplam şatış serisine baktığımızda her ayın sonlarında bir günün bir kırılma noktası olduğunu görüyoruz. Yani satışlar hiç yumuşak yükselişler düşüşler yaşamıyor, çok keskin hareket ediyor. Bu duruma satışların doğal olmasınından çok yılın belirli aylarında yüksek pazarlama bütçeleri ile desteklendiğini düşündürüyor. Çok yükselen her ayın ardından hemen düşmesi bu yükselişin insanları satın almaya yönlendirecek kampanyaların mevcut olduğunu ve bu kampanyalar bitince tüketicilerin de ilgisinin bittiğini ve düşüşün bu yüzden keskin olduğunu düşündürüyor ama tabi farklı bir durum da olabilir.

In [ ]:
!pip install ruptures -qqq

In [ ]:
import ruptures as rpt

rpt_data = temp_data['y'].values.reshape(-1,1)
algo = rpt.Pelt(model = 'l2')
algo.fit(rpt_data)
results = algo.predict(pen = 10)

rpt.display(rpt_data, temp_data.index.to_list(), results)
plt.show()